In [5]:
import os
from autogen import ConversableAgent
from mem0 import MemoryClient
from openai import OpenAI

from dotenv import load_dotenv

/Users/may/.virtualenvs/generative_ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
load_dotenv()

True

In [7]:
USER_ID = "customer_service_bot"

In [8]:
# Initialize Mem0 and AutoGen agents
memory_client = MemoryClient()

In [9]:
agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [{"model": "gpt-4o-2024-08-06"}]},
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [10]:
conversation = [
    {"role": "assistant", "content": "Hi, I'm Best Buy's chatbot! How can I help you?"},
    {"role": "user", "content": "I'm seeing horizontal lines on my TV."},
    {"role": "assistant", "content": "I'm sorry to hear that. Can you provide your TV model?"},
    {"role": "user", "content": "It's a Sony - 77\" Class BRAVIA XR A80K OLED 4K UHD Smart Google TV"},
    {"role": "assistant", "content": "Thank you for the information. Let's troubleshoot this issue..."}
]

memory_client.add(messages=conversation, user_id=USER_ID)
print("Conversation added to memory.")

Conversation added to memory.


In [12]:
question = "What was the issue with my TV?"

In [13]:
relevant_memories = memory_client.search(question, user_id=USER_ID)
relevant_memories

[{'id': '70d8eb78-8051-4f04-b48f-9e194eb2076b',
  'memory': 'User is seeing horizontal lines on their TV',
  'user_id': 'customer_service_bot',
  'hash': '9cec30204bd8d62f76c5d51e5db53a1c',
  'metadata': None,
  'categories': ['technology', 'misc'],
  'created_at': '2024-09-09T19:42:24.696143-07:00',
  'updated_at': '2024-09-09T19:42:24.696158-07:00',
  'custom_categories': None,
  'score': 0.537702862942561},
 {'id': '9bc5c3a3-5cb3-43e3-b411-4e83c5d52d04',
  'memory': 'User has a Sony - 77" Class BRAVIA XR A80K OLED 4K UHD Smart Google TV',
  'user_id': 'customer_service_bot',
  'hash': '206f495b82ad65e1971da343631a5ae3',
  'metadata': None,
  'categories': ['technology', 'entertainment'],
  'created_at': '2024-09-09T19:42:24.663207-07:00',
  'updated_at': '2024-09-09T19:42:24.663224-07:00',
  'custom_categories': None,
  'score': 0.47894369003008896}]

In [11]:
def get_context_aware_response(question):
    relevant_memories = memory_client.search(question, user_id=USER_ID)
    context = "\n".join([m["memory"] for m in relevant_memories])

    prompt = f"""Answer the user question considering the previous interactions:
    Previous interactions:
    {context}

    Question: {question}
    """

    reply = agent.generate_reply(messages=[{"content": prompt, "role": "user"}])
    return reply

# Example usage
answer = get_context_aware_response(question)
print("Context-aware answer:", answer)

Context-aware answer: Based on the previous interactions, the issue with your TV is that you are seeing horizontal lines on your Sony - 77" Class BRAVIA XR A80K OLED 4K UHD Smart Google TV. This can often be due to a hardware problem such as a faulty connection or damage to the display panel, or it could occasionally be related to software issues. To diagnose this effectively, you might want to try troubleshooting steps like checking all cables and connections, updating the TV’s firmware, or performing a factory reset. If the issue persists, contacting Sony support or a professional technician may be necessary.


In [14]:
manager = ConversableAgent(
    "manager",
    system_message="You are a manager who helps in resolving complex customer issues.",
    llm_config={"config_list": [{"model": "gpt-4o-2024-08-06"}]},
    human_input_mode="NEVER"
)

def escalate_to_manager(question):
    relevant_memories = memory_client.search(question, user_id=USER_ID)
    context = "\n".join([m["memory"] for m in relevant_memories])

    prompt = f"""
    Context from previous interactions:
    {context}

    Customer question: {question}

    As a manager, how would you address this issue?
    """

    manager_response = manager.generate_reply(messages=[{"content": prompt, "role": "user"}])
    return manager_response

# Example usage
complex_question = "I'm not satisfied with the troubleshooting steps. What else can be done?"
manager_answer = escalate_to_manager(complex_question)
print("Manager's response:", manager_answer)

Manager's response: I understand your frustration, and I'm here to assist you in resolving this issue as efficiently as possible. Here's what we can do next:

1. **Warranty Check and Service Request**: Let's start by checking if your TV is under warranty. If it is, we can arrange for an authorized technician to inspect and service your TV at no cost to you.

2. **Escalation to Sony Support**: If it is still under warranty and you've already attempted initial troubleshooting steps, it might be beneficial to escalate this issue directly to Sony's support team. They have specialized knowledge and resources for more advanced troubleshooting.

3. **In-Home Service**: Regardless of the warranty status, we can arrange an in-home service where a technician will address the issue on-site. During this visit, they can conduct a thorough diagnostic to determine the cause of the horizontal lines.

4. **Replacement Options**: If the issue cannot be resolved, and it is a persistent defect, we can dis